In [1]:
import pandas as pd
import numpy as np
import os
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

/Users/abhinivesh/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/abhinivesh/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
# source : https://medium.com/@cristhianboujon/how-to-list-the-most-common-words-from-text-corpus-using-scikit-learn-dad4d0cab41d

def get_top_n_words(corpus, n=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    """
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [3]:
unfiltered_na_df = pd.read_csv('FINAL_DATASETS/large_unfiltered_NAs_final.csv')
unfiltered_na_df.head()

,Unnamed: 0,author,created,submissions,comments,daily_downtime
0,0,Ragin33Redhead,2013-04-27 02:45:01+00:00,40,7,15.000000
1,1,OncomingStorm93,2012-12-01 17:49:21+00:00,41,328,9.000000
2,2,SorenBreese,2015-02-21 08:52:48+00:00,1,2,9.503512
3,3,Letterbox10,2014-06-10 23:34:17+00:00,22111,34,6.000000
4,4,ujames,2012-11-13 14:06:26+00:00,299,469,20.000000


In [13]:
normal_accounts_1 = os.listdir('NormalAccounts/')
normal_accounts_2 = os.listdir('NormalAccounts2/')

unwanted_accounts = []

for i in range(len(unfiltered_na_df.author)):
    if unfiltered_na_df.author[i] in normal_accounts_1:
#         print(unfiltered_na_df.author[i], 1)
        pass
    elif unfiltered_na_df.author[i] in normal_accounts_2:
#         print(unfiltered_na_df.author[i], 2)
        pass
    else:
        unwanted_accounts.append(i)
        print(unfiltered_na_df.author[i], 0, i)
        
unfiltered_na_df = unfiltered_na_df.drop(unfiltered_na_df.index[unwanted_accounts])

KellyfromLeedsUK 0 911
KellyfromLeedsUK 0 1138
KellyfromLeedsUK 0 1220


In [28]:
unfiltered_na_df.reset_index(drop=True, inplace=True)

In [31]:
unfiltered_na_df.drop(columns=['Unnamed: 0'], inplace=True)
unfiltered_na_df.to_csv('FINAL_DATASETS/large_unfiltered_NAs_final.csv')

In [33]:
filtered_na_df = pd.read_csv('FINAL_DATASETS/small_filtered_NAs_final.csv')
filtered_na_df.drop(columns=['Unnamed: 0'], inplace=True)
filtered_na_df.head()

,author,created,submissions,comments,daily_downtime
0,KellyStrack,2015-11-02 17:26:44+00:00,133,0,7.000000
1,kabir_chaudhary,2016-08-09 06:15:35+00:00,1,0,9.503512
2,pseph2016,2016-10-29 14:32:02+00:00,4,0,9.503512
3,akunnaila,2016-04-08 02:26:05+00:00,192,0,15.000000
4,alypigcahighve,2015-11-28 07:18:34+00:00,1,0,9.503512


In [34]:
for i in range(len(filtered_na_df.author)):
    if filtered_na_df.author[i] in normal_accounts_1:
#         print(unfiltered_na_df.author[i], 1)
        pass
    elif filtered_na_df.author[i] in normal_accounts_2:
#         print(unfiltered_na_df.author[i], 2)
        pass
    else:
        unwanted_accounts.append(i)
        print(filtered_na_df.author[i], 0, i)
        


In [35]:
filtered_na_df.to_csv('FINAL_DATASETS/small_filtered_NAs_final.csv')

In [36]:
susp_list = os.listdir('SuspiciousAccounts/')
susp_df = pd.read_csv('FINAL_DATASETS/suspicious_accounts.csv')

for i in range(len(susp_df.author)):
    if susp_df.author[i] in susp_list:
#         print(unfiltered_na_df.author[i], 1)
        pass
    else:
        unwanted_accounts.append(i)
        print(susp_df.author[i], 0, i)

In [50]:
def getAllFeatures(accountName, accountType):
    currentPath = os.path.join(accountType, accountName)
    numSubmissions = 0
    numComments = 0
    if 'submissions.csv' in os.listdir(currentPath):
        submissions_csv_path = os.path.join(currentPath, 'submissions.csv')
        submissions_df = pd.read_csv(submissions_csv_path)
        
        uniqueDomains = len(submissions_df['domain'].unique())
        uniqueSubreddits = len(submissions_df['subreddit'].unique())
        uniqueTitles = len(submissions_df['title'].unique())
        numSubmissions = len(submissions_df)
        
        allTitles = ''
        for row in range(len(submissions_df)):
            allTitles += (str(submissions_df.loc[row, 'title']) + ' ')
        top_3_counts = get_top_n_words(allTitles.split(), 3)
        try:
            top_1 = top_3_counts[0][1]
        except:
            top_1 = 0
        try:
            top_2 = top_3_counts[1][1]
        except:
            top_2 = 0
        try:
            top_3 = top_3_counts[2][1]
        except:
            top_3 = 0
#         print(top_1, top_2, top_3)
        
    else:
        uniqueDomains = 0
        uniqueSubreddits = 0
        uniqueTitles = 0
        numSubmissions = 0
        top_1 = 0
        top_2 = 0
        top_3 = 0
        
    if 'comments.csv' in os.listdir(currentPath):
        comments_csv_path = os.path.join(currentPath, 'comments.csv')
        try:
            comments_df = pd.read_csv(comments_csv_path)
            numComments = len(comments_df)
        except:
            numComments = 0
        
        
    else:
        numComments = 0
        
    
        
    if numComments == 0:
        numComments = 1
    if numSubmissions == 0:
        numSubmissions = 1
        
    SubmToCommentRatio = numSubmissions / numComments
    classLabel = 1
    if os.path.dirname(accountType) == 'NormalAccounts':
        classLabel = 0
        
    return [uniqueDomains, uniqueSubreddits, uniqueTitles, numSubmissions, \
           top_1, top_2, top_3, numComments, SubmToCommentRatio, classLabel]

In [46]:
from tqdm import tqdm_notebook as tqdm
def createDataFrame(folderPath):
    
    namesList = os.listdir(folderPath)
    print(folderPath)
    domainsList = []
    subRedditsList = []
    titlesList = []
    submissionsList = []
    top_1List = []
    top_2List = []
    top_3List = []
    commentsList = []
    ratioList = []
    labelsList = []
    
    for accountName in tqdm(namesList):
        featureList = getAllFeatures(accountName, folderPath)
        domainsList.append(featureList[0])
        subRedditsList.append(featureList[1])
        titlesList.append(featureList[2])
        submissionsList.append(featureList[3])
        top_1List.append(featureList[4])
        top_2List.append(featureList[5])
        top_3List.append(featureList[6])
        commentsList.append(featureList[7])
        ratioList.append(featureList[8])
        labelsList.append(featureList[9])
        
    finalDict = {
        'accountName': namesList,
        'uniqueDomains': domainsList,
        'uniqueSubreddits': subRedditsList,
        'uniqueTitles': titlesList,
        'numSubmissions': submissionsList,
        'top_1': top_1List,
        'top_2': top_2List,
        'top_3': top_3List,
        'uniqueComments': commentsList,
        'SubmToCommentRatio' :ratioList,
        'label': labelsList
    }
    
    dataFrame = pd.DataFrame(finalDict)
    return dataFrame

In [48]:
normal_accounts_1_df = createDataFrame('NormalAccounts/')

NormalAccounts/


/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/abh

In [51]:
normal_accounts_2_df = createDataFrame('NormalAccounts2/')
normal_accounts_2_df.label.replace([1], [0], inplace= True)
normal_accounts_1_df.label.replace([1], [0], inplace= True)

NormalAccounts2/


/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/abhin

In [52]:
suspicious_accounts_df = createDataFrame('SuspiciousAccounts/')

SuspiciousAccounts/


In [53]:
temp_df = suspicious_accounts_df.append(normal_accounts_1_df)
temp_df_without_dd = temp_df.append(normal_accounts_2_df)
temp_df_without_dd.shape

(2308, 11)

In [57]:
temp_df_without_dd.reset_index(drop=True, inplace=True)
temp_df_without_dd

,accountName,uniqueDomains,uniqueSubreddits,uniqueTitles,numSubmissions,top_1,top_2,top_3,uniqueComments,SubmToCommentRatio,label
0,ModitiusFaetaur,3,5,8,8,5,2,2,1,8.000000,1
1,DoppleGun,1,13,29,46,19,13,8,1,46.000000,1
2,AgaluneMalordred,2,5,8,8,6,5,5,1,8.000000,1
3,Amohelang,1,1,1,1,1,0,0,1,1.000000,1
4,KulazelKelkree,1,2,5,5,1,1,1,1,5.000000,1
...,...,...,...,...,...,...,...,...,...,...,...
2303,Ozera,39,24,162,164,38,21,16,1185,0.138397,0
2304,dildoodlid,13,14,23,23,8,7,7,251,0.091633,0
2305,cncaudata,8,7,48,50,38,17,15,154,0.324675,0
2306,cs132,9,11,16,16,8,4,3,22,0.727273,0


In [59]:
unfiltered_na_df.rename(columns= {'author': 'accountName'}, inplace=True)
unfiltered_na_df

,accountName,created,submissions,comments,daily_downtime
0,Ragin33Redhead,2013-04-27 02:45:01+00:00,40,7,15.000000
1,OncomingStorm93,2012-12-01 17:49:21+00:00,41,328,9.000000
2,SorenBreese,2015-02-21 08:52:48+00:00,1,2,9.503512
3,Letterbox10,2014-06-10 23:34:17+00:00,22111,34,6.000000
4,ujames,2012-11-13 14:06:26+00:00,299,469,20.000000
...,...,...,...,...,...
2045,Hipsternarwhal74,2016-07-04 00:56:19+00:00,7,6,10.000000
2046,Blubbll,2014-07-01 09:06:05+00:00,980,6199,2.000000
2047,tantalicatom689,2012-02-11 08:17:10+00:00,36,237,19.000000
2048,fattyfattyta,2015-11-05 17:56:50+00:00,1,0,10.000000


In [60]:
susp_df.rename(columns= {'author': 'accountName'}, inplace=True)
susp_df

,Unnamed: 0,accountName,created,submissions,comments,daily_downtime
0,0,BlackToLive,2016-07-05 01:13:18+00:00,28,3,9.000000
1,1,FaurnFlamebreaker,2015-05-15 09:44:41+00:00,1,0,5.816514
2,2,Bill_Jonson,2015-10-17 03:55:40+00:00,1,1,5.816514
3,3,PurebringerOghmagra,2015-05-15 12:38:42+00:00,6,0,5.816514
4,4,Clawisma,2015-09-16 17:22:29+00:00,27,4,3.000000
...,...,...,...,...,...,...
271,271,King_Andersons,2016-03-22 15:55:53+00:00,580,326,4.000000
272,272,BerskyN,2016-10-18 13:43:29+00:00,2,0,5.816514
273,273,WhatImDoindHere,2015-12-29 07:10:38+00:00,180,6,4.000000
274,274,Kevin_Milner,2016-03-30 12:46:36+00:00,2619,285,3.000000


In [61]:
susp_df.drop(columns=['Unnamed: 0'], inplace=True)
temp_with_dd = susp_df.append(unfiltered_na_df)
temp_with_dd

,accountName,created,submissions,comments,daily_downtime
0,BlackToLive,2016-07-05 01:13:18+00:00,28,3,9.000000
1,FaurnFlamebreaker,2015-05-15 09:44:41+00:00,1,0,5.816514
2,Bill_Jonson,2015-10-17 03:55:40+00:00,1,1,5.816514
3,PurebringerOghmagra,2015-05-15 12:38:42+00:00,6,0,5.816514
4,Clawisma,2015-09-16 17:22:29+00:00,27,4,3.000000
...,...,...,...,...,...
2045,Hipsternarwhal74,2016-07-04 00:56:19+00:00,7,6,10.000000
2046,Blubbll,2014-07-01 09:06:05+00:00,980,6199,2.000000
2047,tantalicatom689,2012-02-11 08:17:10+00:00,36,237,19.000000
2048,fattyfattyta,2015-11-05 17:56:50+00:00,1,0,10.000000


In [67]:
final_df_1 = pd.merge(temp_df_without_dd, temp_with_dd, on = 'accountName')
final_df_1.head()

,accountName,uniqueDomains,uniqueSubreddits,uniqueTitles,numSubmissions,top_1,top_2,top_3,uniqueComments,SubmToCommentRatio,label,daily_downtime
0,ModitiusFaetaur,3,5,8,8,5,2,2,1,8.0,1,5.816514
1,DoppleGun,1,13,29,46,19,13,8,1,46.0,1,5.816514
2,AgaluneMalordred,2,5,8,8,6,5,5,1,8.0,1,6.000000
3,Amohelang,1,1,1,1,1,0,0,1,1.0,1,5.816514
4,KulazelKelkree,1,2,5,5,1,1,1,1,5.0,1,5.816514


In [69]:
final_df_2 = final_df_1[['accountName', 'SubmToCommentRatio', 'label', 'daily_downtime']]
final_df_2

,accountName,SubmToCommentRatio,label,daily_downtime
0,ModitiusFaetaur,8.000000,1,5.816514
1,DoppleGun,46.000000,1,5.816514
2,AgaluneMalordred,8.000000,1,6.000000
3,Amohelang,1.000000,1,5.816514
4,KulazelKelkree,5.000000,1,5.816514
...,...,...,...,...
2321,Ozera,0.138397,0,2.000000
2322,dildoodlid,0.091633,0,12.000000
2323,cncaudata,0.324675,0,14.000000
2324,cs132,0.727273,0,10.000000


In [70]:
final_df_1.to_csv('FINAL_DATASETS/all_accounts_unfiltered_1.csv')
final_df_2.to_csv('FINAL_DATASETS/all_accounts_unfiltered_2.csv')

In [72]:
filtered_na_df.rename(columns= {'author': 'accountName'}, inplace=True)
temp_with_dd_2 = susp_df.append(filtered_na_df)
temp_with_dd_2

,accountName,created,submissions,comments,daily_downtime
0,BlackToLive,2016-07-05 01:13:18+00:00,28,3,9.000000
1,FaurnFlamebreaker,2015-05-15 09:44:41+00:00,1,0,5.816514
2,Bill_Jonson,2015-10-17 03:55:40+00:00,1,1,5.816514
3,PurebringerOghmagra,2015-05-15 12:38:42+00:00,6,0,5.816514
4,Clawisma,2015-09-16 17:22:29+00:00,27,4,3.000000
...,...,...,...,...,...
271,reticulated_python,2012-08-01 14:05:27+00:00,56,983,8.000000
272,Zamoon,2015-03-12 12:46:48+00:00,48,106,8.000000
273,jrwhite8,2010-08-22 08:20:01+00:00,1,885,14.000000
274,Sean3ezy,2013-08-19 05:19:19+00:00,188,951,3.000000


In [73]:
temp_with_dd_2.drop(columns=['created', 'submissions', 'comments'], inplace=True)
temp_with_dd_2.reset_index(drop=True, inplace=True)
temp_with_dd_2

,accountName,daily_downtime
0,BlackToLive,9.000000
1,FaurnFlamebreaker,5.816514
2,Bill_Jonson,5.816514
3,PurebringerOghmagra,5.816514
4,Clawisma,3.000000
...,...,...
547,reticulated_python,8.000000
548,Zamoon,8.000000
549,jrwhite8,14.000000
550,Sean3ezy,3.000000


In [74]:
final_df_3 = pd.merge(temp_with_dd_2, temp_df_without_dd, on = 'accountName')
final_df_3

,accountName,daily_downtime,uniqueDomains,uniqueSubreddits,uniqueTitles,numSubmissions,top_1,top_2,top_3,uniqueComments,SubmToCommentRatio,label
0,BlackToLive,9.000000,7,14,20,28,22,19,13,3,9.333333,1
1,FaurnFlamebreaker,5.816514,1,1,1,1,1,1,1,1,1.000000,1
2,Bill_Jonson,5.816514,1,1,1,1,1,1,1,1,1.000000,1
3,PurebringerOghmagra,5.816514,2,2,6,6,3,3,2,1,6.000000,1
4,Clawisma,3.000000,2,17,27,27,9,7,5,4,6.750000,1
...,...,...,...,...,...,...,...,...,...,...,...,...
547,reticulated_python,8.000000,21,17,54,56,28,27,15,983,0.056968,0
548,Zamoon,8.000000,9,8,48,48,33,25,14,106,0.452830,0
549,jrwhite8,14.000000,1,1,1,1,1,1,1,885,0.001130,0
550,Sean3ezy,3.000000,16,16,145,188,71,62,41,951,0.197687,0


In [75]:
final_df_4 = final_df_3[['accountName', 'SubmToCommentRatio', 'label', 'daily_downtime']]
final_df_4

,accountName,SubmToCommentRatio,label,daily_downtime
0,BlackToLive,9.333333,1,9.000000
1,FaurnFlamebreaker,1.000000,1,5.816514
2,Bill_Jonson,1.000000,1,5.816514
3,PurebringerOghmagra,6.000000,1,5.816514
4,Clawisma,6.750000,1,3.000000
...,...,...,...,...
547,reticulated_python,0.056968,0,8.000000
548,Zamoon,0.452830,0,8.000000
549,jrwhite8,0.001130,0,14.000000
550,Sean3ezy,0.197687,0,3.000000


In [76]:
final_df_3.to_csv('FINAL_DATASETS/all_accounts_filtered_1.csv')
final_df_4.to_csv('FINAL_DATASETS/all_accounts_filtered_2.csv')

In [77]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
# <br /><br /> ==> after the above steps, we are getting "br br"
# we are including them into stop words list
# instead of <br /> if we have <br/> these tags would have revmoved in the 1st step

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
 "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
'because', 'as', 'until', 'while', 'of', \
 'at', 'by', 'for', 'with', 'about', 'against', 'between',
'into', 'through', 'during', 'before', 'after',\
 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out',
'on', 'off', 'over', 'under', 'again', 'further',\
 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
 's', 't', 'can', 'will', 'just', 'don', "don't", 'should',
"should've", 'now', 'd', 'll', 'm', 'o', 're', \
 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't",
'didn', "didn't", 'doesn', "doesn't", 'hadn',\
 "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
 "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn',
"shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
 'won', "won't", 'wouldn', "wouldn't"])

# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
 # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
 # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def clean_string(sentence):
    sentence = decontracted(sentence)
    sentence = re.sub("\S*\d\S*", "", sentence).strip()
    sentence = re.sub('[^A-Za-z]+', ' ', sentence)
    # https://gist.github.com/sebleier/554280
    sentence = ' '.join(e.lower() for e in sentence.split() if e.lower() not in stopwords)
    return sentence.strip()

In [78]:
all_titles_list = []
all_comments_list = []
account_names_list = []

susp_names_list = os.listdir('SuspiciousAccounts/')
normal_names_list = os.listdir('NormalAccounts/')
normal_names_list_2 = os.listdir('NormalAccounts2/')

for eachName in tqdm(susp_names_list):
    currentPath = os.path.join('SuspiciousAccounts', eachName)
    all_titles = ''
    all_comments = ''
    
    if 'submissions.csv' in os.listdir(currentPath):
        submissions_csv_path = os.path.join(currentPath, 'submissions.csv')
        submissions_df = pd.read_csv(submissions_csv_path)
        for row in range(len(submissions_df)):
            all_titles += (submissions_df.loc[row, 'title'] + ' ')
    
    if 'comments.csv' in os.listdir(currentPath):
        comments_csv_path = os.path.join(currentPath, 'comments.csv')
        comments_df = pd.read_csv(comments_csv_path)
        for row in range(len(comments_df)):
            all_comments += (comments_df.loc[row, 'body'] + ' ')
    
    all_titles_list.append(clean_string(all_titles))
    all_comments_list.append(clean_string(all_comments))
    account_names_list.append(eachName)
    
for eachName in tqdm(normal_names_list):
    currentPath = os.path.join('NormalAccounts', eachName)
    all_titles = ''
    all_comments = ''
    
    if 'submissions.csv' in os.listdir(currentPath):
        submissions_csv_path = os.path.join(currentPath, 'submissions.csv')
        submissions_df = pd.read_csv(submissions_csv_path)
        for row in range(len(submissions_df)):
            all_titles += (submissions_df.loc[row, 'title'] + ' ')
    
    if 'comments.csv' in os.listdir(currentPath):
        comments_csv_path = os.path.join(currentPath, 'comments.csv')
        comments_df = pd.read_csv(comments_csv_path)
        for row in range(len(comments_df)):
            all_comments += (str(comments_df.loc[row, 'body']) + ' ')
    
    all_titles_list.append(clean_string(all_titles))
    all_comments_list.append(clean_string(all_comments))
    account_names_list.append(eachName)
    
for eachName in tqdm(normal_names_list_2):
    currentPath = os.path.join('NormalAccounts2', eachName)
    all_titles = ''
    all_comments = ''
    
    if 'submissions.csv' in os.listdir(currentPath):
        submissions_csv_path = os.path.join(currentPath, 'submissions.csv')
        try:
            submissions_df = pd.read_csv(submissions_csv_path)
            for row in range(len(submissions_df)):
                all_titles += (submissions_df.loc[row, 'title'] + ' ')
        except:
            all_titles = ''
    
    if 'comments.csv' in os.listdir(currentPath):
        comments_csv_path = os.path.join(currentPath, 'comments.csv')
        try:
            comments_df = pd.read_csv(comments_csv_path)
            for row in range(len(comments_df)):
                all_comments += (str(comments_df.loc[row, 'body']) + ' ')
        except:
            all_comments = ''
    
    all_titles_list.append(clean_string(all_titles))
    all_comments_list.append(clean_string(all_comments))
    account_names_list.append(eachName)

/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [82]:
# Function to Transform Sentences to Average Word2Vec Vectors with 50 dimensions

def transform_sent_to_avg_w2v(sentences, w2v_model, word_dimensions=20):
    sent_vectors = []
    w2v_words = list(w2v_model.wv.vocab) # save list of unique vocab
    for sent in tqdm(sentences): # for each review/sentence
        sent_vec = np.zeros(word_dimensions) # as word vectors are of zero length 50, you might need to change this to 300 if you use google'sw2v
        cnt_words =0; # num of words with a valid vector in the sentence/review
        for word in sent.split(): # for each word in a review/sentence
            if word in w2v_words:
                vec = w2v_model.wv[word]
                sent_vec += vec
                cnt_words += 1
        if cnt_words != 0:
            sent_vec /= cnt_words
        sent_vectors.append(sent_vec)
    return sent_vectors

In [80]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
w2v_model_titles = Word2Vec([all_titles.split() for all_titles in all_titles_list], min_count=5, size=20, workers=4)
w2v_model_comments = Word2Vec([all_comments.split() for all_comments in all_comments_list], min_count=5, size=20, workers=4)

/Users/abhinivesh/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [83]:
all_titles_w2v = transform_sent_to_avg_w2v(all_titles_list, w2v_model_titles)
all_comments_w2v = transform_sent_to_avg_w2v(all_comments_list, w2v_model_comments)

In [84]:
titles_colnames = []
comments_colnames = []

for i in range(1, 21):
    titles_colnames.append('title_w2v_' + str(i))
    comments_colnames.append('comment_w2v_' + str(i))

print(titles_colnames)
print(comments_colnames)

['title_w2v_1', 'title_w2v_2', 'title_w2v_3', 'title_w2v_4', 'title_w2v_5', 'title_w2v_6', 'title_w2v_7', 'title_w2v_8', 'title_w2v_9', 'title_w2v_10', 'title_w2v_11', 'title_w2v_12', 'title_w2v_13', 'title_w2v_14', 'title_w2v_15', 'title_w2v_16', 'title_w2v_17', 'title_w2v_18', 'title_w2v_19', 'title_w2v_20']
['comment_w2v_1', 'comment_w2v_2', 'comment_w2v_3', 'comment_w2v_4', 'comment_w2v_5', 'comment_w2v_6', 'comment_w2v_7', 'comment_w2v_8', 'comment_w2v_9', 'comment_w2v_10', 'comment_w2v_11', 'comment_w2v_12', 'comment_w2v_13', 'comment_w2v_14', 'comment_w2v_15', 'comment_w2v_16', 'comment_w2v_17', 'comment_w2v_18', 'comment_w2v_19', 'comment_w2v_20']


In [85]:
w2v_df = pd.DataFrame(np.array(all_titles_w2v), columns= titles_colnames)
w2v_df_comments = pd.DataFrame(np.array(all_comments_w2v), columns= comments_colnames)
for col in w2v_df_comments.columns:
    w2v_df[col] = w2v_df_comments[col]
w2v_df['accountName'] = account_names_list
w2v_df.head()

,title_w2v_1,title_w2v_2,title_w2v_3,title_w2v_4,title_w2v_5,title_w2v_6,title_w2v_7,title_w2v_8,title_w2v_9,title_w2v_10,...,comment_w2v_12,comment_w2v_13,comment_w2v_14,comment_w2v_15,comment_w2v_16,comment_w2v_17,comment_w2v_18,comment_w2v_19,comment_w2v_20,accountName
0,-1.449525,0.252968,-1.655974,-0.951808,-0.307696,0.435586,1.415624,-0.460771,-0.662506,1.537867,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ModitiusFaetaur
1,-2.256031,1.152917,-1.568804,-0.273519,0.162308,0.797661,1.901837,-0.711790,-0.292550,0.991454,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DoppleGun
2,-1.886414,0.994590,-1.605252,-0.404225,0.038070,0.859134,2.160536,-0.421546,-0.743668,1.191616,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AgaluneMalordred
3,-5.274011,3.135416,-4.721818,-6.049441,-0.837588,1.790578,7.090599,-1.507640,0.419691,1.041431,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Amohelang
4,-1.756332,0.686284,-2.018741,-0.711321,-0.057473,0.127160,1.691476,-0.735132,-1.192029,1.311052,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,KulazelKelkree


In [86]:
final_df_5 = pd.merge(final_df_1, w2v_df, on= 'accountName')
final_df_5

,accountName,uniqueDomains,uniqueSubreddits,uniqueTitles,numSubmissions,top_1,top_2,top_3,uniqueComments,SubmToCommentRatio,...,comment_w2v_11,comment_w2v_12,comment_w2v_13,comment_w2v_14,comment_w2v_15,comment_w2v_16,comment_w2v_17,comment_w2v_18,comment_w2v_19,comment_w2v_20
0,ModitiusFaetaur,3,5,8,8,5,2,2,1,8.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,DoppleGun,1,13,29,46,19,13,8,1,46.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,AgaluneMalordred,2,5,8,8,6,5,5,1,8.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Amohelang,1,1,1,1,1,0,0,1,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,KulazelKelkree,1,2,5,5,1,1,1,1,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2321,Ozera,39,24,162,164,38,21,16,1185,0.138397,...,-1.669937,-0.835594,-0.177706,-1.002143,-0.340881,-1.003435,0.270984,0.852472,-0.899248,0.129830
2322,dildoodlid,13,14,23,23,8,7,7,251,0.091633,...,-1.409016,-0.328016,-0.192373,-1.079826,-0.201413,-1.082248,0.044926,1.017636,-0.816537,0.840475
2323,cncaudata,8,7,48,50,38,17,15,154,0.324675,...,-1.878195,-0.976064,-0.420159,-0.960458,-0.614904,-1.596627,0.522190,1.009904,-0.619775,-0.286724
2324,cs132,9,11,16,16,8,4,3,22,0.727273,...,-1.884541,-0.792592,-0.140178,-1.498555,-0.469695,-0.748387,0.890630,1.187701,-1.025432,-0.028427


In [88]:
final_df_6 = pd.merge(final_df_3, w2v_df, on= 'accountName')
final_df_6

,accountName,daily_downtime,uniqueDomains,uniqueSubreddits,uniqueTitles,numSubmissions,top_1,top_2,top_3,uniqueComments,...,comment_w2v_11,comment_w2v_12,comment_w2v_13,comment_w2v_14,comment_w2v_15,comment_w2v_16,comment_w2v_17,comment_w2v_18,comment_w2v_19,comment_w2v_20
0,BlackToLive,9.000000,7,14,20,28,22,19,13,3,...,-1.099268,-0.442361,0.742166,-1.300467,-0.195409,-0.859859,0.893082,1.207561,-1.201036,0.465572
1,FaurnFlamebreaker,5.816514,1,1,1,1,1,1,1,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bill_Jonson,5.816514,1,1,1,1,1,1,1,1,...,-2.278738,-0.504375,0.255763,-0.568763,-0.549338,-1.280800,-0.575678,1.627818,-1.824830,0.495577
3,PurebringerOghmagra,5.816514,2,2,6,6,3,3,2,1,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Clawisma,3.000000,2,17,27,27,9,7,5,4,...,-2.013705,-0.781464,-0.998365,-1.277605,-0.477285,-0.392368,-0.718539,0.565686,-2.138833,-0.424110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,reticulated_python,8.000000,21,17,54,56,28,27,15,983,...,-1.896306,-0.763239,0.115544,-1.192535,-0.279991,-0.984891,0.066436,1.028707,-0.663200,0.299756
548,Zamoon,8.000000,9,8,48,48,33,25,14,106,...,-1.708901,-0.534607,-0.107640,-0.982506,-0.484530,-0.858957,0.055576,0.719055,-0.792342,0.130892
549,jrwhite8,14.000000,1,1,1,1,1,1,1,885,...,-0.604248,0.010252,0.405322,-0.742970,0.265783,-1.138160,0.313297,1.177109,-1.015299,0.770748
550,Sean3ezy,3.000000,16,16,145,188,71,62,41,951,...,-1.473829,-1.009802,-0.245797,-1.006575,-0.786085,-1.368034,0.471624,1.004550,-0.691355,0.097938


In [89]:
final_df_5.to_csv('FINAL_DATASETS/all_accounts_unfiltered_w2v.csv')
final_df_6.to_csv('FINAL_DATASETS/all_accounts_filtered_w2v.csv')